In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import datasets
from tqdm.notebook import tqdm

import utils

# Load pilot

In [2]:
score_column = 'label'
annotations_file = './khalil_pilots.csv'
database = 'pcmasterrace'
within = 0.2

In [3]:
# pilot_df = pd.read_csv(annotations_file, delimiter='\t', header=None)
# pilot_df.columns = ['bin', 'link', 'comment', 'raw_label']
# pilot_df = pilot_df.dropna()

df = pd.read_csv(annotations_file, header=[0])
df.columns = ['random', 'database', 'bin', 'link', 'comment', 'raw_label', 'notes']
df[score_column] = [ 1 if i == 'y' else 0 for i in df['raw_label'] ]
df.head(2)

,random,database,bin,link,comment,raw_label,notes,label
0,0.000134,askreddit,"(-0.00043000000000000004, 0.000737]",https://www.reddit.com/r/AskReddit/comments/zq...,"Case in point, try a Danish Blue on steak. Tan...",n,NaN,0
1,0.000660,askreddit,"(-0.00043000000000000004, 0.000737]",https://www.reddit.com/r/askreddit/comments/zk...,It was actually hailed as 'last mile' transpor...,n,NaN,0


In [4]:
df.groupby('database').count()

,random,bin,link,comment,raw_label,notes,label
database,,,,,,,
askreddit,400,400,400,400,400,0,400
legaladvice,400,400,400,400,400,1,400
pcmasterrace,400,400,400,400,400,0,400
politics,400,400,400,400,400,0,400
sex,400,400,400,400,400,3,400
wow,400,400,400,400,400,1,400


In [5]:
pilot_df = df[df.database == database]

# Pilot estimate of p

In [6]:
pilot_mean = 0.
for i, group in pilot_df.groupby('bin'):
    strata_mean = group[score_column].mean()
    pilot_mean += strata_mean * (len(group) / len(pilot_df))
pilot_mean

0.04

# Allocation

In [7]:
sizes_sigmas = []
for i, group in pilot_df.groupby('bin'):
    pilot = np.array(group[score_column])
    # laplace smoothing
    pilot = np.concatenate([np.array([1,0]), pilot])
    print(pilot.sum(), i)
    sizes_sigmas.append((len(group), np.std(pilot)))
print()

size = 5000
allocations = []
denominator = np.sum([ n_h * sigma_h for n_h, sigma_h in sizes_sigmas ])
for n_h, sigma_h in sizes_sigmas:
    n_from_bin = size * n_h * sigma_h / denominator
    print(n_h, sigma_h, n_from_bin)
    allocations.append(n_from_bin)

1 (-0.00043599999999999997, 0.000666]
1 (0.000666, 0.000734]
1 (0.000734, 0.000844]
2 (0.000844, 0.00107]
1 (0.00107, 0.00176]
2 (0.00176, 0.00706]
3 (0.00706, 0.763]
13 (0.763, 0.999]

50 0.13733516208736252 429.1376695179857
50 0.13733516208736252 429.1376695179857
50 0.13733516208736252 429.1376695179857
50 0.19230769230769232 600.9129319322293
50 0.13733516208736252 429.1376695179857
50 0.1923076923076923 600.9129319322292
50 0.23316068563427197 728.5682103222597
50 0.4330127018922193 1353.0552477413391


In [8]:
minimum = np.min(allocations)
multipliers = [ i / minimum for i in allocations ]
multipliers

[1.0,
 1.0,
 1.0,
 1.4002800840280096,
 1.0,
 1.4002800840280094,
 1.6977493752543307,
 3.152963125472328]

In [9]:
stratified_var = 0
for (i, group), n_from_bin in zip(pilot_df.groupby('bin'), allocations):
    print(i, len(group), n_from_bin, group[score_column].mean(), group[score_column].var())
    
    p = group[score_column].mean()
    
    # approximation when the groups are very large
    stratified_var += np.square(len(group) / len(pilot_df)) * (group[score_column].var() / n_from_bin)
'stddev of estimator: ', np.sqrt(stratified_var)

(-0.00043599999999999997, 0.000666] 50 429.1376695179857 0.0 0.0
(0.000666, 0.000734] 50 429.1376695179857 0.0 0.0
(0.000734, 0.000844] 50 429.1376695179857 0.0 0.0
(0.000844, 0.00107] 50 600.9129319322293 0.02 0.019999999999999976
(0.00107, 0.00176] 50 429.1376695179857 0.0 0.0
(0.00176, 0.00706] 50 600.9129319322292 0.02 0.019999999999999955
(0.00706, 0.763] 50 728.5682103222597 0.04 0.0391836734693878
(0.763, 0.999] 50 1353.0552477413391 0.24 0.18612244897959182


('stddev of estimator: ', 0.0020074249003956914)

In [10]:
stddev = np.sqrt(stratified_var)
left = stats.norm.ppf([0.025]).item() * stddev + pilot_mean
right = stats.norm.ppf([0.975]).item() * stddev + pilot_mean
print('[%.4f, %.4f]' % (left, right))

[0.0361, 0.0439]


In [11]:
numerator = 0
for (i, group), multip_h in zip(pilot_df.groupby('bin'), multipliers):
    print(i, len(group), multip_h, group[score_column].mean(), group[score_column].var())
    
    # approximation when the groups are very large
    numerator += np.square(len(group) / len(pilot_df)) * (group[score_column].var() / multip_h)

(-0.00043599999999999997, 0.000666] 50 1.0 0.0 0.0
(0.000666, 0.000734] 50 1.0 0.0 0.0
(0.000734, 0.000844] 50 1.0 0.0 0.0
(0.000844, 0.00107] 50 1.4002800840280096 0.02 0.019999999999999976
(0.00107, 0.00176] 50 1.0 0.0 0.0
(0.00176, 0.00706] 50 1.4002800840280094 0.02 0.019999999999999955
(0.00706, 0.763] 50 1.6977493752543307 0.04 0.0391836734693878
(0.763, 0.999] 50 3.152963125472328 0.24 0.18612244897959182


# Power analysis

In [12]:
def calculate_n(var, desired_ci, alpha=0.05):
    z_statistic = stats.norm.ppf(1 - (alpha / 2))
    se_squared = np.square(desired_ci / z_statistic)
    n = var / se_squared
    return n

In [13]:
rs_n = []
for p in [left, right]:
    desired_ci = p * within     # within 5 percent of p
    var = p * (1 - p)

    rs_n.append(calculate_n(var, desired_ci)+1)
    print(p, int(rs_n[-1]))

0.03606551949355554 2567
0.04393448050644446 2090


In [14]:
ss_n = []
for p in [left, right]:
    desired_ci = p * within

    alpha = 0.05
    z_statistic = stats.norm.ppf(1 - (alpha / 2))
    desired_var = np.square(desired_ci / z_statistic)

    minimum = numerator / desired_var
    ss_n.append(np.sum([minimum * multip for multip in multipliers])+1)
    print(p, int(ss_n[-1]))

0.03606551949355554 1488
0.04393448050644446 1003


In [15]:
desired_ci = 0.005

alpha = 0.05
z_statistic = stats.norm.ppf(1 - (alpha / 2))
desired_var = np.square(desired_ci / z_statistic)

minimum = numerator / desired_var
mde_n = np.sum([minimum * multip for multip in multipliers])+1
print(p, int(mde_n))

0.04393448050644446 3097


In [16]:
efficiency = 1 - (ss_n[0] / rs_n[0])
efficiency

0.4202622486267441

In [17]:
print('r/%s & %.2f\%% & [%.2f\%%, %.2f\%%] & [%d, %d] & [%d, %d] & %d\%% & %d \\\\' % (database, pilot_mean*100, left*100, right*100, ss_n[1], ss_n[0], rs_n[1], rs_n[0], efficiency*100, mde_n,))

r/pcmasterrace & 4.00\% & [3.61\%, 4.39\%] & [1003, 1488] & [2090, 2567] & 42\% & 3097 \\


In [18]:
pilot_mean, np.round(left, 4), np.round(right, 4)

(0.04, 0.0361, 0.0439)